In [1]:
import numpy as np
import pyci
import psi4

In [6]:
# Lets intiate AOint() a subclass of psi4utils() 
aoint = pyci.utils.AOint('cc-pvdz', '../.xyz/Be.xyz')
# saves all the necessary integrals as .npz files
aoint.save_all_aoints()
aoint.save_ao_dpints()
aoint.save_ao_qdints()
# run scf and get info for canonical HF orbitals
aoint.save_mo_info()
# lets load  ao_erints, mo_info and try to get the CIS matrix
ao_erints = np.load('.scratch/ao_erints.npz')['electron_repulsion_aoints'] 
eps_a = np.load('.scratch/mo_scf_info.npz')['eps_a']
Ca = np.load('.scratch/mo_scf_info.npz')['Ca'] 
ao_dipoles = np.load('.scratch/ao_dpints.npz')
ao_quadrupoles = np.load('.scratch/ao_qdints.npz')
# lets convert our erints from AO basis to MO basis
mo_erints = aoint.eri_ao2mo(Ca, ao_erints, greedy=True)
# mo_dpx = aoint.matrix_ao2mo(Ca, ao_dipoles['dpx_aoints'])
# mo_dpy = aoint.matrix_ao2mo(Ca, ao_dipoles['dpy_aoints'])
# mo_dpz = aoint.matrix_ao2mo(Ca, ao_dipoles['dpz_aoints'])
# mo_qdxx = aoint.matrix_ao2mo(Ca, ao_quadrupoles['qdxx_aoints'])
# mo_qdyy = aoint.matrix_ao2mo(Ca, ao_quadrupoles['qdyy_aoints'])
# mo_qdzz = aoint.matrix_ao2mo(Ca, ao_quadrupoles['qdzz_aoints'])
del ao_erints, ao_dipoles, ao_quadrupoles

Size of the ERI tensor will be 0.00 Gb

Ground state SCF Energy : -14.57233763 

Nuclear repulsion energy : 0.00000000 

Total electronic energy : -14.57233763 

!!!Warning: Using greedy eri_ao2mo transform



In [7]:
nbf, nmo, nso, na, nb, nocc, nvirt = aoint.get_orb_info(aoint.wfn)

 ORBITAL INFORMATION  
-----------------------
Basis functions   : 14  
Molecular orbitals: 14  
Spin Orbitals     : 28  
Alpha Orbitals    : 2  
Beta Orbitals     : 2  
Occupied Orbitals : 2  
Virtual Orbitals  : 12  



In [8]:
scf_energy = aoint.scf_energy

In [9]:
orbinfo = (nocc, nmo)
active_space = (nocc,nvirt)
options = {'singles' : True,
            'full_cis' : True,
            'doubles' : True,
            'doubles_iiaa' : True,
            'doubles_iiab' : True,
            'doubles_ijaa' : True,
            'doubles_ijab_A' : True,
            'doubles_ijab_B' : True}
csfs, num_csfs = pyci.configint.rcisd.generate_csfs(orbinfo, active_space, options)
num_csfs, sum(num_csfs)


([1, 24, 24, 132, 12, 66, 66], 325)

In [10]:
type(csfs)

list

In [11]:
HCISD = pyci.configint.rcisd.comp_hcisd(eps_a, mo_erints, scf_energy, orbinfo, active_space, options, ncore=4)

In [12]:
HCISD_c = pyci.configint.rcisd.c_comp_hcisd(eps_a, mo_erints, scf_energy, orbinfo, active_space, options, ncore=4)

AttributeError: module 'pyci.lib.configint.rcisd' has no attribute 'comp_hrow_hf'

In [ ]:
HCISD.shape

In [ ]:
# Comparing pyci with psi4
from scipy.sparse.linalg import eigsh
eval_cisd, evec_cisd = eigsh(HCISD0, k=1, which='SM')
psi4_cisd = psi4.energy('CISD')
print("pyci CISD E0: {e:16.16f}".format(e=vals[0]))
print("psi4 CISD E0: {e:16.16f}".format(e=psi4_cisd))
print("dE : {dE:1.2E}".format(dE=abs(psi4_cisd-vals[0])))

In [ ]:
# Computing all the eigen values
HCISD0 = HCISD - np.eye(sum(num_csfs))*scf_energy
vals, vecs = np.linalg.eigh(HCISD0)
vals += scf_energy 